In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from transformer_multistep_type import train_start
import torch

In [2]:
params = {'input_window' : [120],
         'output_window' : [1],
         'batch_size' : [32],
         'epochs' : [150],
         'feature_size' : [16, 32, 64, 128, 256],
         'd_ff' : [128, 256, 512, 1024],
         'num_layers' : [1, 2],
         'lr' : [5e-4],
         'dropout' : [0.1]}

In [3]:
cnt = 0
total = 1
for param in params:
    total *= len(params[param])


def select_param(params, param_names, selected_params=[], param_idx=0):
    if param_idx == 9:
        torch.cuda.empty_cache()
        global cnt
        cnt += 1
        print(f"Start training ({cnt} / {total})")
        for param_name, params in zip(param_names, selected_params):
            print(f"{param_name} : {params}", end= ' | ')
        print()
        train_start(*selected_params)
        return
    
    param_name = param_names[param_idx]
    param_list = params[param_name]
    for parameter in param_list:
        select_param(params, param_names, selected_params + [parameter], param_idx + 1)


In [4]:
import importlib
import transformer_multistep_type
importlib.reload(transformer_multistep_type)
from transformer_multistep_type import train_start

In [5]:
select_param(params, list(params.keys()))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Start training (1 / 40)
input_window : 120 | output_window : 1 | batch_size : 32 | epochs : 150 | feature_size : 16 | d_ff : 128 | num_layers : 1 | lr : 0.0005 | dropout : 0.1 | 


wandb: Currently logged in as: qja1998. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
100%|██████████| 150/150 [06:37<00:00,  2.65s/it]

Start training (2 / 40)
input_window : 120 | output_window : 1 | batch_size : 32 | epochs : 150 | feature_size : 16 | d_ff : 128 | num_layers : 2 | lr : 0.0005 | dropout : 0.1 | 


...

pred_MAE,▃▆▃▃█▃▆▁▆▄
pred_MSE,▃▆▃▃█▃▆▁▆▄
pred_SMAPE,█▆▁▄█▆▃▃▃▃
val_loss_A,█▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_B,█▂▄▁▃▁▁▁▁▁▁▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁
val_loss_C,▃▂█▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_D,▄▅▂█▂▃▂▃▄▃▁▁▂▂▂▂▂▁▃▂▂▃▂▁▁▁▂▁▃▂▁▂▁▁▁▁▁▂▁▁
val_loss_E,█▃█▂▁▂▄▂▁▂▅▃▃▂▂▆▅▁▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
val_loss_G,▂█▄▁▃▂▂▂▁▂▂▁▁▁▃▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂
val_loss_H,█▂▂▂▁▁▁▁▁▂▁▂▂▁▁▁▁▃▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▂▁▂▁▁▁▁
val_loss_I,█▄▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


100%|██████████| 150/150 [07:04<00:00,  2.83s/it]

Start training (40 / 40)
input_window : 120 | output_window : 1 | batch_size : 32 | epochs : 150 | feature_size : 256 | d_ff : 1024 | num_layers : 2 | lr : 0.0005 | dropout : 0.1 | 


pred_MAE,▄▃▁█▁▁▁▄▂▄
pred_MSE,▄▃▁█▁▁▁▄▂▄
pred_SMAPE,▁▄▃█▆▁▃▄▃▃
val_loss_A,█▂▁▁▁▂▁▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_B,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_C,▃▄█▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_D,▇█▅▁▁▂▁▂▁▁▃▁▂▁▁▁▂▂▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_E,█▇▃▅▁▁▂▄▄▁▁▁▂▄▁▁▂▂▂▁▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▂
val_loss_G,▃▄█▃▂▂▁▃▃▁▁▂▂▁▂▂▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_H,█▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_I,█▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


100%|██████████| 150/150 [11:51<00:00,  4.74s/it]
